# 3D Gaussian Splatting Training Pipeline
### Professional Implementation for Research

**Author:** Sumit maheshwari  
**Date:** February 2026  
**Description:** Complete pipeline for training 3D Gaussian Splatting models from video data

---

## Pipeline Architecture
1. **Data Upload & Validation**
2. **COLMAP Structure-from-Motion (SfM)**
3. **3D Gaussian Splatting Training**
4. **Model Export (PLY format)**
5. **Visualization & Quality Metrics**

## 1. Environment Setup

In [1]:
# Check GPU availability
!nvidia-smi

# Mount Google Drive for data persistence
from google.colab import drive
drive.mount('/content/drive')

/bin/bash: line 1: nvidia-smi: command not found


ValueError: mount failed

In [2]:
# Install dependencies
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install plyfile tqdm pillow opencv-python

# Install COLMAP
!apt-get install -y colmap

# Clone 3D Gaussian Splatting repository
!git clone https://github.com/graphdeco-inria/gaussian-splatting --recursive
%cd gaussian-splatting

# Install submodules
!pip install submodules/diff-gaussian-rasterization
!pip install submodules/simple-knn

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 1.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  at-spi2-core gsettings-desktop-schemas libamd2 libatk-bridge2.0-0
  libatk1.0-0 libatk1.0-data libatspi2.0-0 libcamd2 libccolamd2 libceres2
  libcholmod3 libcolamd2 libcxsparse3 libdouble-conversion3 libevdev2
  libfreeimage3 libgflags2.2 libglew2.2 libgoogle-glog0v5 libgtk-3-0
  libgtk-3-bin libgtk-3-common libgudev-1.0-0 libilmbase25 libinput-bin
  libinput10 libjxr0 libmd4c0 libmetis5 libmtdev1 libopenexr25 libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libraw20
  librsvg2-common libspqr2 libsuitesparseconfig5 libwacom-bin libwacom-common
  libwacom9 libxcb-icccm4 libxcb-image0 libxcb-keysyms1 libxcb-render-util0
  libxcb-util1 libxcb-xinerama0 libxcb-xinput0 libxcb

## 2. Data Upload & Validation

In [3]:
import os
import json
import shutil
from pathlib import Path
from google.colab import files
import zipfile

# Configuration
PROJECT_NAME = "gs_experiment_001"
DATA_ROOT = f"training_data"
OUTPUT_ROOT = f"/data/output"

# Create directories
os.makedirs(DATA_ROOT, exist_ok=True)
os.makedirs(OUTPUT_ROOT, exist_ok=True)

print(f"Data directory: {DATA_ROOT}")
print(f"Output directory: {OUTPUT_ROOT}")

Data directory: training_data
Output directory: /data/output


In [4]:
# Upload preprocessed data (zip file containing extracted frames)
# Option 1: Upload from local machine
uploaded = files.upload()

# Extract uploaded zip
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(DATA_ROOT)
        print(f"Extracted {filename} to {DATA_ROOT}")

# Option 2: Copy from Google Drive (uncomment if using)
# DRIVE_DATA_PATH = "/content/drive/MyDrive/your_data.zip"
# shutil.copy(DRIVE_DATA_PATH, DATA_ROOT)

# Validate data structure
images_dir = os.path.join(DATA_ROOT, "images")
if os.path.exists(images_dir):
    num_images = len([f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png'))])
    print(f"✓ Found {num_images} images in {images_dir}")
else:
    print("✗ Images directory not found. Please check your upload.")

KeyboardInterrupt: 

## 3. COLMAP Structure-from-Motion

COLMAP performs:
- Feature extraction
- Feature matching
- Sparse reconstruction
- Camera pose estimation

In [ ]:
# COLMAP configuration
COLMAP_DB = os.path.join(DATA_ROOT, "database.db")
SPARSE_DIR = os.path.join(DATA_ROOT, "sparse")
os.makedirs(SPARSE_DIR, exist_ok=True)

print("Starting COLMAP SfM pipeline...")
print("This may take 10-30 minutes depending on the number of images.")

In [ ]:
# Feature extraction
!colmap feature_extractor \
    --database_path {COLMAP_DB} \
    --image_path {images_dir} \
    --ImageReader.single_camera 1 \
    --ImageReader.camera_model OPENCV \
    --SiftExtraction.use_gpu 1

In [ ]:
# Feature matching
!colmap exhaustive_matcher \
    --database_path {COLMAP_DB} \
    --SiftMatching.use_gpu 1

In [ ]:
# Sparse reconstruction (mapper)
!colmap mapper \
    --database_path {COLMAP_DB} \
    --image_path {images_dir} \
    --output_path {SPARSE_DIR}

In [ ]:
# Convert COLMAP binary to text format (for inspection)
SPARSE_MODEL_DIR = os.path.join(SPARSE_DIR, "0")
SPARSE_TEXT_DIR = os.path.join(DATA_ROOT, "sparse_text")

!colmap model_converter \
    --input_path {SPARSE_MODEL_DIR} \
    --output_path {SPARSE_TEXT_DIR} \
    --output_type TXT

# Display reconstruction statistics
cameras_txt = os.path.join(SPARSE_TEXT_DIR, "cameras.txt")
images_txt = os.path.join(SPARSE_TEXT_DIR, "images.txt")
points3D_txt = os.path.join(SPARSE_TEXT_DIR, "points3D.txt")

if os.path.exists(points3D_txt):
    with open(points3D_txt, 'r') as f:
        num_points = sum(1 for line in f if not line.startswith('#') and line.strip())
    print(f"\n✓ COLMAP reconstruction successful!")
    print(f"  - Reconstructed 3D points: {num_points}")
else:
    print("\n✗ COLMAP reconstruction failed. Check the logs above.")

## 4. 3D Gaussian Splatting Training

In [ ]:
# Training configuration
ITERATIONS = 30000  # Standard: 30k iterations
MODEL_PATH = os.path.join(OUTPUT_ROOT, "output")

# Hyperparameters (can be tuned for your specific use case)
POSITION_LR_INIT = 0.00016
POSITION_LR_FINAL = 0.0000016
FEATURE_LR = 0.0025
OPACITY_LR = 0.05
SCALING_LR = 0.005
ROTATION_LR = 0.001

print(f"Training for {ITERATIONS} iterations")
print(f"Model will be saved to: {MODEL_PATH}")

In [ ]:
%cd /content/gaussian-splatting

# Run training
!python train.py \
    -s {DATA_ROOT} \
    -m {MODEL_PATH} \
    --iterations {ITERATIONS} \
    --position_lr_init {POSITION_LR_INIT} \
    --position_lr_final {POSITION_LR_FINAL} \
    --feature_lr {FEATURE_LR} \
    --opacity_lr {OPACITY_LR} \
    --scaling_lr {SCALING_LR} \
    --rotation_lr {ROTATION_LR} \
    --save_iterations 7000 15000 30000 \
    --checkpoint_iterations 7000 15000 30000

## 5. Model Export & Validation

In [ ]:
# Locate the final PLY file
FINAL_PLY = os.path.join(MODEL_PATH, "point_cloud", f"iteration_{ITERATIONS}", "point_cloud.ply")

if os.path.exists(FINAL_PLY):
    file_size_mb = os.path.getsize(FINAL_PLY) / (1024 * 1024)
    print(f"✓ Model successfully saved: {FINAL_PLY}")
    print(f"  File size: {file_size_mb:.2f} MB")
    
    # Copy to Google Drive for persistence
    drive_ply = os.path.join(OUTPUT_ROOT, f"{PROJECT_NAME}_final.ply")
    shutil.copy(FINAL_PLY, drive_ply)
    print(f"✓ Copied to Google Drive: {drive_ply}")
else:
    print("✗ PLY file not found. Training may have failed.")

In [ ]:
# Analyze the PLY file
from plyfile import PlyData

if os.path.exists(FINAL_PLY):
    plydata = PlyData.read(FINAL_PLY)
    num_gaussians = len(plydata['vertex'])
    
    print("\n=== Model Statistics ===")
    print(f"Number of Gaussians: {num_gaussians:,}")
    print(f"Properties: {plydata['vertex'].data.dtype.names}")
    
    # Calculate model complexity metrics
    import numpy as np
    positions = np.stack([plydata['vertex']['x'], 
                         plydata['vertex']['y'], 
                         plydata['vertex']['z']], axis=1)
    
    bbox_min = positions.min(axis=0)
    bbox_max = positions.max(axis=0)
    scene_extent = np.linalg.norm(bbox_max - bbox_min)
    
    print(f"\nScene Extent: {scene_extent:.3f} units")
    print(f"Bounding Box:")
    print(f"  Min: {bbox_min}")
    print(f"  Max: {bbox_max}")

## 6. Rendering & Visualization

In [ ]:
# Render validation views
RENDER_OUTPUT = os.path.join(OUTPUT_ROOT, "renders")

!python render.py \
    -m {MODEL_PATH} \
    --iteration {ITERATIONS} \
    --skip_train \
    --skip_test

print(f"\nRendered images saved to: {os.path.join(MODEL_PATH, 'test')}")

In [ ]:
# Display sample renders
import matplotlib.pyplot as plt
from PIL import Image
import glob

render_dir = os.path.join(MODEL_PATH, "test", f"ours_{ITERATIONS}", "renders")
render_files = sorted(glob.glob(os.path.join(render_dir, "*.png")))[:6]

if render_files:
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for idx, img_path in enumerate(render_files):
        img = Image.open(img_path)
        axes[idx].imshow(img)
        axes[idx].axis('off')
        axes[idx].set_title(f"View {idx+1}")
    
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_ROOT, "render_preview.png"), dpi=150, bbox_inches='tight')
    plt.show()
    print(f"Preview saved to: {os.path.join(OUTPUT_ROOT, 'render_preview.png')}")
else:
    print("No render images found.")

## 7. Quality Metrics & Evaluation

In [ ]:
# Compute metrics (PSNR, SSIM, LPIPS)
!python metrics.py \
    -m {MODEL_PATH}

In [ ]:
# Load and display metrics
metrics_file = os.path.join(MODEL_PATH, "results.json")

if os.path.exists(metrics_file):
    with open(metrics_file, 'r') as f:
        metrics = json.load(f)
    
    print("\n=== Quality Metrics ===")
    for key, value in metrics.items():
        if isinstance(value, dict):
            print(f"\n{key}:")
            for subkey, subvalue in value.items():
                print(f"  {subkey}: {subvalue:.4f}")
        else:
            print(f"{key}: {value}")
else:
    print("Metrics file not found.")

## 8. Export Final Results

In [ ]:
# Create comprehensive export package
import datetime

EXPORT_DIR = os.path.join(OUTPUT_ROOT, "export")
os.makedirs(EXPORT_DIR, exist_ok=True)

# Copy essential files
shutil.copy(FINAL_PLY, os.path.join(EXPORT_DIR, "model.ply"))

if os.path.exists(metrics_file):
    shutil.copy(metrics_file, os.path.join(EXPORT_DIR, "metrics.json"))

# Create experiment report
report = {
    "experiment_name": PROJECT_NAME,
    "date": datetime.datetime.now().isoformat(),
    "training_iterations": ITERATIONS,
    "num_input_images": num_images,
    "num_gaussians": num_gaussians if 'num_gaussians' in locals() else "N/A",
    "model_size_mb": file_size_mb if 'file_size_mb' in locals() else "N/A",
    "hyperparameters": {
        "position_lr_init": POSITION_LR_INIT,
        "position_lr_final": POSITION_LR_FINAL,
        "feature_lr": FEATURE_LR,
        "opacity_lr": OPACITY_LR,
        "scaling_lr": SCALING_LR,
        "rotation_lr": ROTATION_LR
    }
}

with open(os.path.join(EXPORT_DIR, "experiment_report.json"), 'w') as f:
    json.dump(report, f, indent=2)

print(f"\n✓ Export package created at: {EXPORT_DIR}")
print("\nContents:")
for item in os.listdir(EXPORT_DIR):
    print(f"  - {item}")

In [ ]:
# Download final PLY file
from google.colab import files

final_export_ply = os.path.join(EXPORT_DIR, "model.ply")
if os.path.exists(final_export_ply):
    print("Downloading PLY file...")
    files.download(final_export_ply)
    print("Download complete!")
else:
    print("PLY file not found for download.")

---

## Summary

This notebook provides a complete pipeline for:
1. ✓ Data validation and preparation
2. ✓ COLMAP Structure-from-Motion
3. ✓ 3D Gaussian Splatting training
4. ✓ PLY model export
5. ✓ Quality metrics evaluation
6. ✓ Visualization and rendering

**Next Steps for Publication:**
- Run ablation studies with different hyperparameters
- Compare against baseline methods (NeRF, etc.)
- Conduct user studies for perceptual quality
- Document computational requirements and runtime
- Generate figures and visualizations for paper

**Citation:**
```
@article{kerbl20233d,
  title={3D Gaussian Splatting for Real-Time Radiance Field Rendering},
  author={Kerbl, Bernhard and Kopanas, Georgios and Leimk{\"u}hler, Thomas and Drettakis, George},
  journal={ACM Transactions on Graphics},
  volume={42},
  number={4},
  year={2023}
}
```